In [1]:
# Importing the libraries
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from chembl_webresource_client.new_client import new_client
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score
import pickle


In [2]:
target = new_client.target
target_query = target.search('3-hydroxy-3-methylglutaryl-coenzyme A').only(['target_chembl_id','pref_name','organism','target_type'])
df = pd.DataFrame(target_query)
display(df)

,organism,pref_name,target_chembl_id,target_type
0,Homo sapiens,HMG-CoA reductase,CHEMBL402,SINGLE PROTEIN
1,Rattus norvegicus,HMG-CoA reductase,CHEMBL3247,SINGLE PROTEIN
2,Mus musculus,HMG-CoA reductase,CHEMBL2764,SINGLE PROTEIN
3,Mesocricetus auratus,3-hydroxy-3-methylglutaryl-coenzyme A reductase,CHEMBL1075267,SINGLE PROTEIN
4,Schizosaccharomyces pombe 972h-,3-hydroxy-3-methylglutaryl-coenzyme A reductase,CHEMBL1163121,SINGLE PROTEIN
...,...,...,...,...
7547,Canis lupus familiaris,Protein kinase C (PKC),CHEMBL5303570,PROTEIN FAMILY
7548,Homo sapiens,Estradiol 17-beta-dehydrogenase 11,CHEMBL5305043,SINGLE PROTEIN
7549,Mus musculus,17-beta-hydroxysteroid dehydrogenase 13,CHEMBL5305044,SINGLE PROTEIN
7550,Variola virus,Envelope phospholipase OPG057,CHEMBL5308522,SINGLE PROTEIN


In [3]:
selected_target = df['target_chembl_id'][1]
selected_target

'CHEMBL3247'

In [8]:
dict_df = new_client.activity.filter(target_chembl_id=selected_target)
df = pd.DataFrame.from_dict(dict_df)
display(df)

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,105945,[],CHEMBL691797,In vitro inhibitory activity was measured agai...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,uM,UO_0000065,None,130.0
1,None,None,105946,[],CHEMBL850314,The compound was tested for the relative poten...,B,None,None,BAO_0000179,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,Relative potency,None,None,None,0.00015
2,None,None,105962,[],CHEMBL692517,In vitro for inhibition of rat HMG-CoA reductase.,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,I50,nM,UO_0000065,None,19.1
3,None,None,105999,[],CHEMBL688751,Inhibitory activity against HMG-CoA reductase ...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,nM,UO_0000065,None,13.0
4,None,None,106011,[],CHEMBL691934,Inhibition of HMG-CoA reductase activity in pa...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,uM,UO_0000065,None,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,None,297732,17678562,[],CHEMBL3706257,Reductase Assay: All assays were carried out i...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,nM,UO_0000065,None,3.0
1647,None,297733,17678563,[],CHEMBL3706257,Reductase Assay: All assays were carried out i...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,nM,UO_0000065,None,1.0
1648,None,297734,17678564,[],CHEMBL3706257,Reductase Assay: All assays were carried out i...,B,None,None,BAO_0000190,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,IC50,nM,UO_0000065,None,1.0
1649,"{'action_type': 'INHIBITOR', 'description': 'N...",Not Active,23344382,"[{'comments': None, 'relation': None, 'result_...",CHEMBL4882521,HMGCOARED Eurofins-Panlabs selectivity data (BI),B,None,None,BAO_0000179,...,Rattus norvegicus,HMG-CoA reductase,10116,None,None,% Ctrl,%,UO_0000187,None,87.0


In [9]:
df['type'].value_counts()

type
IC50                873
Relative potency    322
Inhibition          217
I50                 108
Log IC50             77
Activity             29
Ratio                 7
Decrease              7
Concentration         5
Ki                    2
% Ctrl                2
Km                    1
Vmax                  1
Name: count, dtype: int64

In [5]:
df = df[df['type'] == 'Ki']
df.reset_index(drop=True,inplace=True)

df = df[["molecule_chembl_id","canonical_smiles","standard_value","standard_units"]]
df


,molecule_chembl_id,canonical_smiles,standard_value,standard_units
0,CHEMBL54440,CC[C@H](C)C(=O)O[C@H]1CCC=C2C=C[C@H](C)[C@H](C...,1.4,nM
1,CHEMBL54440,CC[C@H](C)C(=O)O[C@H]1CCC=C2C=C[C@H](C)[C@H](C...,1.0,nM
